In [1]:
root_nq_files = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\NATURAL_QUESTIONS\\csvs'
nq_doc_text_path = root_nq_files + '\\nq_doc_text_chunck.csv'
nq_long_answer_path = root_nq_files + '\\nq_long_answer_chunck.csv'
nq_short_answer_path = root_nq_files + '\\nq_short_answer_chunck.csv'
nq_question_path = root_nq_files + '\\nq_question_chunck.csv'
nq_infos_path = root_nq_files + '\\nq_infos_chunck.csv'


In [2]:
import jsonlines
import pdb
import re
import pandas as pd
import math

In [3]:
nq_doc_text_df = pd.read_csv(nq_doc_text_path)  
nq_long_answer_df = pd.read_csv(nq_long_answer_path)  
nq_short_answer_df = pd.read_csv(nq_short_answer_path)  
nq_question_df = pd.read_csv(nq_question_path)  
nq_infos_df = pd.read_csv(nq_infos_path)  

In [6]:
ids_unicos = nq_infos_df['ID'].unique()

In [7]:
nq_translated_list = [] 
for id_unico in ids_unicos:
    #infos
    try:
        document_url = nq_infos_df.loc[nq_infos_df['ID'] == id_unico]['DOCUMENT_URL'].values[0]
        yes_no_answer = nq_infos_df.loc[nq_infos_df['ID'] == id_unico]['YES_NO_ANSWER'].values[0]
    except:
        import pdb;pdb.set_trace()

    #Question Text
    question = nq_question_df.loc[nq_question_df['ID'] == id_unico]['QUESTION'].values[0]
    # Document Text
    doc_text = ''
    nq_doc_text_df_id_unico = nq_doc_text_df.loc[nq_doc_text_df['ID'] == id_unico]
    for i,row in nq_doc_text_df_id_unico.iterrows():
        doc_text = doc_text + row['DOC_TEXT'] + ' '
    doc_text = doc_text.lstrip()
    
    # Long Answer Text
    long_answer_text = ''
    long_answer_text_id_unico = nq_long_answer_df.loc[nq_long_answer_df['ID'] == id_unico]
    for i,row in long_answer_text_id_unico.iterrows():
        long_answer_text += row['LONG_ANSWER']
    long_answer_text = long_answer_text.lstrip()
    
    if long_answer_text:
        start_la = long_answer_text_id_unico['START_LA'].values[0]
        end_la = long_answer_text_id_unico['END_LA'].values[0]
        long_answer_dict = {'long_answer':long_answer_text,'start_token':start_la ,'end_token':end_la}
    else:
        long_answer_dict = {}
    
    # Short Answer Text
    short_answers_list = []
    short_answer_text_id_unico = nq_short_answer_df.loc[nq_short_answer_df['ID'] == id_unico]
    for i,row in short_answer_text_id_unico.iterrows():
        dict_short_ansewr = {'short_answer':row['SHORT_ANSWER'], 'start_token':row['START_SA'],'end_token':row['END_SA'] }
        short_answers_list.append(dict_short_ansewr)
        
        
    
    nq_translated =   {
            "example_id":id_unico ,
            "document_url": document_url,
            "question_text": question,
            "document_text": doc_text,
            "annotations": [{
              "long_answer": long_answer_dict,
              "short_answers": short_answers_list,
              "yes_no_answer": yes_no_answer,
            }]
        }
    nq_translated_list.append(nq_translated)


In [ ]:
with jsonlines.open('portuguese_natural_questions.jsonl', mode='w') as writer:
    for row_dict in nq_translated_list:
        writer.write(row_dict)